In [0]:
from google.colab import files
files.upload()

In [0]:
!pip install -q kaggle

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
!mkdir -p ~/.kaggle

In [0]:
cp kaggle.json ~/.kaggle/

In [8]:
!kaggle datasets download -d jessicali9530/celeba-dataset

 99% 1.21G/1.21G [00:11<00:00, 156MB/s]
100% 1.21G/1.21G [00:11<00:00, 111MB/s]


In [9]:
!unzip celeba-dataset.zip

Archive:  celeba-dataset.zip
  inflating: list_attr_celeba.csv    
  inflating: img_align_celeba.zip    
  inflating: list_landmarks_align_celeba.csv  
  inflating: list_bbox_celeba.csv    
  inflating: list_eval_partition.csv  


In [10]:
ls

adc.json              kaggle.json           list_eval_partition.csv
celeba-dataset.zip    list_attr_celeba.csv  list_landmarks_align_celeba.csv
img_align_celeba.zip  list_bbox_celeba.csv  sample_data/


In [0]:
!unzip img_align_celeba.zip

In [0]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras import Input
from keras.applications import VGG19
from keras.callbacks import TensorBoard
from keras.layers import BatchNormalization, Activation, LeakyReLU, Add, Dense, PReLU, Flatten
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array, load_img


In [0]:
def residual_block(x):
  
  res = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(x)
  res = Activation(activation = "relu")(res)
  res = BatchNormalization(momentum = 0.8)(res)
  
  res = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(x)
  res = BatchNormalization(momentum = 0.8)(res)
  
  res = Add()([res, x])
  
  return res

In [0]:
def build_gen():
  
  res_blocks = 16
  input_shape = (64, 64, 3)
  
  input_layer = Input(shape = input_shape)
  
  gen1 = Conv2D(filters = 64, kernel_size = 9, strides = 1, padding = 'same', activation = 'relu')(input_layer)
  
  res = residual_block(gen1)
  for i in range(res_block - 1):
    res = residual_block(res)
    
  gen2 = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(res)
  gen2 = BatchNormalization(momentum = 0.8)(gen2)
  
  gen3 = Add()([gen2, gen1])
  
  gen4 = UpSampling2D(size = (2, 2))(gen3)
  gen4 = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = "same")(gen4)
  gen4 = Activation('relu')(gen4)
  
  gen5 = UpSampling2D(size = (2, 2))(gen4)
  gen5 = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = "same")(gen5)
  gen5 = Activation('relu')(gen5)
  
  gen6 = Conv2D(filters = 3, kernel_size = 9, strides = 1, padding = "same")(gen5)
  output = Activation('tanh')(gen6)
  
  model = Model(inputs = [input_layer], outputs = [output], name = 'generator')
  
  return model